## Ajustando o ambiente de trabalho

- Definição da sessão spark
- Construção da conexão com o drive

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=5b37946ab224fb653bde09bea9df9838f14f01d74b956524d47c9d8f96c4bf95
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [2]:
# definindo a sessão spark
from pyspark.sql import SparkSession 

spark = SparkSession.builder\
                    .master('local[*]')\
                    .appName('Challenge DataScience - 2 Ed')\
                    .getOrCreate()
spark

In [3]:
# criando a conexão com o drive
from google.colab import drive
drive.mount('my-drive')

Mounted at my-drive


## Iniciando o processo de análise e tratamento
- Leitura da base de dados
- Verificação da quantidade de linhas e colunas 
- Avaliação da estrutura da base de dados
- Verificação da coerência dos datatypes com respeito ao [dicionário de dados](https://github.com/millenagena/Challenge-Data-Science-Alura-2ed/blob/main/1-Transformando-os-dados/dados/README.md#dicion%C3%A1rio-de-dados---anuncio) 

In [4]:
dataset = spark.read.json(
    path='/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset_bruto.json'
)

dataset.show(5)

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [5]:
dataset.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [6]:
dataset.count()

89083

In [7]:
89083-293

88790

- Até aqui podemos concluir que estamos lidando com uma base de aproximadamente 90000 registros e com uma quantidade razoável de colunas. De acordo com a regra de negócio que nos foi passada, somente os registros dos campos de "anuncio" do arquivo json lido serão relevantes. Portanto, efetuo atualizações no dataframe de tal maneira estruturar essas colunas da melhor maneira possível

In [8]:
dataset\
  .select(
      'anuncio.id',
      'anuncio.tipo_anuncio',
      'anuncio.tipo_unidade',
      'anuncio.tipo_uso',
      'anuncio.area_total',
      'anuncio.area_util',
      'anuncio.quartos',
      'anuncio.suites',
      'anuncio.banheiros',
      'anuncio.vaga',
      'anuncio.andar',
      'anuncio.endereco',
      'anuncio.valores'
  ).show(10)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+--------------------+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|            endereco|             valores|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+--------------------+
|47d553e0-79f2-4a4...|       Usado|      Outros|  Comercial|        []|     [16]|    [0]|   [0]|      [0]| [1]|    0|{Centro, 20061003...|[{260, 107, Venda...|
|b6ffbae1-17f6-487...|       Usado|      Outros|  Comercial|        []|     [14]|    [0]|    []|      [0]| [0]|    0|{Centro, 20051040...|[{260, 107, Venda...|
|1fb030a5-9e3e-4a1...|       Usado|      Outros|  Comercial|    [1026]|   [1026]|    [0]|    []|      [0]| [0]|    0|{Maria da Graça, ...|[{null, 1613, Ven...|
|ac6cb348-69d6-45a...|       Usado|     

In [9]:
dataset = dataset\
            .select(
                'anuncio.id',
                'anuncio.tipo_anuncio',
                'anuncio.tipo_unidade',
                'anuncio.tipo_uso',
                'anuncio.area_total',
                'anuncio.area_util',
                'anuncio.quartos',
                'anuncio.suites',
                'anuncio.banheiros',
                'anuncio.vaga',
                'anuncio.andar',
                'anuncio.endereco',
                'anuncio.valores'
            )

- Do select anterior podemos notar que alguns campos: area_total, area_util, quartos, suites, banheiros, vaga, caracteristicas e valores estão estruturados com registros do tipo array. Queremos que esses dados se torem inteiros ao invés de listas para, posteriormente, codificar nosso modelo de previsão.

- Outro ponto que merece atenção também é: o campo de endereço contém registros ainda encadeados (marcas do formato json original), enquanto o campo valores contém também registros encadeados, mas dentro de uma lista. Queremos efetuar o tratamento desses campos e transformá-los em colunas únicas também.

In [10]:
# antes, verifico se as colunas numéricas anteriores em formato array contém registros com mais de um valor (não faz sentido) e, 
# se sim, quantos são
from pyspark.sql import functions as f

# o primeiro argumento desses arrays tem índice 0, então verifico quantos registros possuem algum elemento não nulo com índice 1
# (i.e. quantos registros não fazem sentido)

# 1) análise do campo area_total
dadosInconsistentes_area_total = dataset.where(f.col('area_total').getItem(1).isNotNull()).count()

# 2) análise do campo area_util
dadosInconsistentes_area_util = dataset.where(f.col('area_util').getItem(1).isNotNull()).count()

# 3) análise do campo quartos
dadosInconsistentes_quartos = dataset.where(f.col('quartos').getItem(1).isNotNull()).count()

# 4) análise do campo suites
dadosInconsistentes_suites = dataset.where(f.col('suites').getItem(1).isNotNull()).count()

# 5) análise do campo banheiros
dadosInconsistentes_banheiros = dataset.where(f.col('banheiros').getItem(1).isNotNull()).count()

# 6) análise do campo vaga
dadosInconsistentes_vaga = dataset.where(f.col('vaga').getItem(1).isNotNull()).count()

In [11]:
print(dadosInconsistentes_area_total)
print(dadosInconsistentes_area_util)
print(dadosInconsistentes_quartos)
print(dadosInconsistentes_suites)
print(dadosInconsistentes_banheiros)
print(dadosInconsistentes_vaga)

0
224
160
103
112
94


In [12]:
# cálculo do total de registros inconsistentes contabilizando todas as colunas avaliadas
dataset\
            .select('*')\
            .filter(f.col('area_total').getItem(1).isNotNull() \
                    | f.col('area_util').getItem(1).isNotNull() \
                    | f.col('quartos').getItem(1).isNotNull() \
                    | f.col('suites').getItem(1).isNotNull() \
                    | f.col('banheiros').getItem(1).isNotNull() \
                    | f.col('vaga').getItem(1).isNotNull()
                    ).count()

239

- Podemos ver que cada uma das colunas avaliadas (cujas informações são importantes para o desenvolvimento do modelo) contém uma pequena quantidade de registros inconsistentes / inválidos, ou seja, com uma informação extra no array que não temos como saber de onde é.

- Entretanto, vale destacar que, mesmo contabilizando todas as quantidades de registros inconsistentes encontrados para cada uma das colunas, obtemos um total de apenas 239 dados. Essa quantidade de dados representa uma parcela muito pequena dos ~90000 da base com a qual estamos trabalhando. Não perderemos praticamente nada se eliminarmos esses dados e seguirmos com a modelagem do problema.

_(ps.: a quantidade de linhas inconsistentes encontradas se dá pelo fato de que, muito provavelmente, as linhas (tuplas) que contém registros inconsistentes para uma dada coluna, também contém registros inconsistentes para os demais.)_


- Antes de efetuar essa exclusão verifico, para essas colunas analisadas, se não existem dados com um eventual terceiro valor (índice 2) dentro dos arrays (apenas por garantia).

In [13]:
# 1) análise do campo area_total
dadosInconsistentes_area_total = dataset.where(f.col('area_total').getItem(2).isNotNull()).count()

# 2) análise do campo area_util
dadosInconsistentes_area_util = dataset.where(f.col('area_util').getItem(2).isNotNull()).count()

# 3) análise do campo quartos
dadosInconsistentes_quartos = dataset.where(f.col('quartos').getItem(2).isNotNull()).count()

# 4) análise do campo suites
dadosInconsistentes_suites = dataset.where(f.col('suites').getItem(2).isNotNull()).count()

# 5) análise do campo banheiros
dadosInconsistentes_banheiros = dataset.where(f.col('banheiros').getItem(2).isNotNull()).count()

# 6) análise do campo vaga
dadosInconsistentes_vaga = dataset.where(f.col('vaga').getItem(2).isNotNull()).count()

print(dadosInconsistentes_area_total)
print(dadosInconsistentes_area_util)
print(dadosInconsistentes_quartos)
print(dadosInconsistentes_suites)
print(dadosInconsistentes_banheiros)
print(dadosInconsistentes_vaga)

0
0
0
0
0
0


- Aparentamente existem apenas aqueles poucos arrays com dois valores. Não parece haver nenhum array com 3 (ou mais) valores.

In [14]:
dataset.count()

89083

In [15]:
# efetuo a exclusão dos dados avaliados, definindo o dataset apenas com as linhas
# que POSSUEM DADO NULO no 2 item dos arrays

dataset = dataset\
              .select('*')\
              .filter(
                    f.col('area_total').getItem(1).isNull() \
                    & f.col('area_util').getItem(1).isNull() \
                    & f.col('quartos').getItem(1).isNull() \
                    & f.col('suites').getItem(1).isNull() \
                    & f.col('banheiros').getItem(1).isNull() \
                    & f.col('vaga').getItem(1).isNull())

# ps.: a negação dos "ou's" do teste anterior se torna "e's"

In [16]:
dataset.count()

88844

- Resultado consistente com o valor esperado. Tínhamos 239 dados defeituosos de acordo com nossas verificações anteriores.

$$
89083 - 239 = 88844
$$

In [17]:
dataset.show(5)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+--------------------+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|            endereco|             valores|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+--------------------+
|47d553e0-79f2-4a4...|       Usado|      Outros|  Comercial|        []|     [16]|    [0]|   [0]|      [0]| [1]|    0|{Centro, 20061003...|[{260, 107, Venda...|
|b6ffbae1-17f6-487...|       Usado|      Outros|  Comercial|        []|     [14]|    [0]|    []|      [0]| [0]|    0|{Centro, 20051040...|[{260, 107, Venda...|
|1fb030a5-9e3e-4a1...|       Usado|      Outros|  Comercial|    [1026]|   [1026]|    [0]|    []|      [0]| [0]|    0|{Maria da Graça, ...|[{null, 1613, Ven...|
|ac6cb348-69d6-45a...|       Usado|     

- Ainda falta eliminar o caráter array das colunas supracitadas e extrair as informações encadeadas dos campos endereco e valores.

In [18]:
# efetuo um select para validar os resultados
dataset\
            .select(
                'id',
                'tipo_anuncio',
                'tipo_unidade',
                'tipo_uso',
                f.col('area_total').getItem(0).alias('area_total'),
                f.col('area_util').getItem(0).alias('area_util'),
                f.col('quartos').getItem(0).alias('quartos'),
                f.col('suites').getItem(0).alias('suites'),
                f.col('banheiros').getItem(0).alias('banheiros'),
                f.col('vaga').getItem(0).alias('vaga'),
                'andar',
                'endereco.*',
                f.col('valores').getItem(0).alias('valores')
            ).show(5)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------+--------+--------------+--------------+----------+----------+----+--------------------+------------+--------------------+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|        bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|        zona|             valores|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------+--------+--------------+--------------+----------+----------+----+--------------------+------------+--------------------+
|47d553e0-79f2-4a4...|       Usado|      Outros|  Comercial|      null|       16|      0|     0|        0|   1|    0|        Centro|20061003|Rio de Janeiro|Rio de Janeiro|-22.906082| -43.18671|  BR|    Rua Buenos Aires|Zona Central|{260, 

In [19]:
# aplico as atualizações do select anterior
dataset = dataset\
            .select(
                'id',
                'tipo_anuncio',
                'tipo_unidade',
                'tipo_uso',
                f.col('area_total').getItem(0).alias('area_total'),
                f.col('area_util').getItem(0).alias('area_util'),
                f.col('quartos').getItem(0).alias('quartos'),
                f.col('suites').getItem(0).alias('suites'),
                f.col('banheiros').getItem(0).alias('banheiros'),
                f.col('vaga').getItem(0).alias('vaga'),
                'andar',
                'endereco.*',
                f.col('valores').getItem(0).alias('valores')
            )

In [20]:
# incluo as colunas encadeadas no campo valores e excluo a coluna correspondente (trazida de forma repetido pelo '*')
dataset = dataset\
              .select('*', 'valores.*')
  
dataset = dataset.drop('valores')

In [21]:
dataset.show(5)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------+--------+--------------+--------------+----------+----------+----+--------------------+------------+----------+----+-----+-----+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|        bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|        zona|condominio|iptu| tipo|valor|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------+--------+--------------+--------------+----------+----------+----+--------------------+------------+----------+----+-----+-----+
|47d553e0-79f2-4a4...|       Usado|      Outros|  Comercial|      null|       16|      0|     0|        0|   1|    0|        Centro|20061003|Rio de Janeiro|Rio de Janeiro|-22.906082| -43.18671|  BR|    Rua Buenos Aire

In [22]:
dataset.printSchema()

root
 |-- id: string (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- andar: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- pais: string (nullable = true)
 |-- rua: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



- Por fim, podemos notar que nem todas as colunas estão apresentando o tipo correto de dado. Por exemplo: area_total, area_utl, iptu, condominio... 
Todas essas colunas (e algumas outras mais) deveriam ter dados no formato inteiro, mas estão com dados no formato string.

In [23]:
# efetuo o cast de conversão para o data type mais adequado

cols_to_correct = [
    'area_total',
    'area_util',
    'quartos',
    'suites',
    'banheiros',
    'cep',
    'condominio',
    'iptu',
    'valor'
]

from pyspark.sql.types import LongType

for col in cols_to_correct:
  dataset = dataset.withColumn( col, f.col(col).cast(LongType()) )

In [24]:
dataset.printSchema()

root
 |-- id: string (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- area_total: long (nullable = true)
 |-- area_util: long (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- andar: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: long (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- pais: string (nullable = true)
 |-- rua: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: long (nullable = true)
 |-- iptu: long (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: long (nullable = true)



In [25]:
dataset.show(5)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------+--------+--------------+--------------+----------+----------+----+--------------------+------------+----------+----+-----+-----+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|        bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|        zona|condominio|iptu| tipo|valor|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------+--------+--------------+--------------+----------+----------+----+--------------------+------------+----------+----+-----+-----+
|47d553e0-79f2-4a4...|       Usado|      Outros|  Comercial|      null|       16|      0|     0|        0|   1|    0|        Centro|20061003|Rio de Janeiro|Rio de Janeiro|-22.906082| -43.18671|  BR|    Rua Buenos Aire

- De acordo com a regra de negócio do projeto, é necessário efetuar um filtro nas colunas de tipos. Precisamos do seguinte:
  - tipo_uso: residencial
  - tipo_unidade: apartamento
  - tipo_anuncio: usado

In [26]:
# para avaliar o possível impacto que esses filtros terão no modelo, faço uma contagem de frequência
# associada a cada um desses tipos

dataset.groupBy('tipo_uso', 'tipo_unidade', 'tipo_anuncio').count().show()

+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio|count|
+-----------+------------+------------+-----+
|Residencial|      Outros|       Usado| 7502|
|Residencial| Apartamento|  Lançamento|   17|
|Residencial| Apartamento|       Usado|66562|
|  Comercial| Apartamento|       Usado|    4|
|  Comercial|      Outros|       Usado| 4443|
|  Comercial|        Casa|       Usado|   92|
|Residencial|        Casa|       Usado|10224|
+-----------+------------+------------+-----+



- Temos cerca de 60000 dados com as características requisitadas (~ 60% da base)

In [27]:
'''
dataset = dataset\
              .select('*')\
              .filter(
                    f.col('tipo_uso') == 'Residencial' \
                    & f.col('tipo_unidade') == 'Apartamento' \
                    & f.col('tipo_anuncio') == 'Usado'
                    )
'''

"\ndataset = dataset              .select('*')              .filter(\n                    f.col('tipo_uso') == 'Residencial'                     & f.col('tipo_unidade') == 'Apartamento'                     & f.col('tipo_anuncio') == 'Usado'\n                    )\n"

In [28]:
# farei o filtro dessas condições 1 por 1 porque, por algum motivo, o spark 
# tá dando erro colocando todos juntos
dataset = dataset.select('*').filter(f.col('tipo_uso') == 'Residencial')
dataset = dataset.select('*').filter(f.col('tipo_unidade') == 'Apartamento')
dataset = dataset.select('*').filter(f.col('tipo_anuncio') == 'Usado')

In [29]:
dataset.groupBy('tipo_uso', 'tipo_unidade', 'tipo_anuncio').count().show()

+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio|count|
+-----------+------------+------------+-----+
|Residencial| Apartamento|       Usado|66562|
+-----------+------------+------------+-----+



- Ainda de acordo com a regra de negócio, foi solicitado que somente as colunas de bairro e zona fossem mantidas (com relação ao endereço)

In [30]:
dataset = dataset.drop('cep')\
                 .drop('cidade')\
                 .drop('estado')\
                 .drop('latitude')\
                 .drop('longitude')\
                 .drop('pais')\
                 .drop('rua')

- Ainda de acordo com a regra de negócio, devemos manter apenas os registros que contenham o tipo "venda" registrado na coluna "tipo" (proveniente do campo "valores" no json originalmente lido)

In [31]:
dataset.groupBy('tipo').count().show()

+-------+-----+
|   tipo|count|
+-------+-----+
|Aluguel|  214|
|  Venda|66348|
+-------+-----+



In [32]:
# de acordo com o select feito anteriormente, a maior parte dos nossos dados foram cadastrados com esse tipo classificação
# (então não teremos muitos problemas com perda de informação)
dataset = dataset.filter(f.col('tipo') == 'Venda')

In [33]:
dataset.show(5)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+---------+----------+----------+----+-----+-----+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|   bairro|      zona|condominio|iptu| tipo|valor|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+---------+----------+----------+----+-----+-----+
|d2e3a3aa-09b5-45a...|       Usado| Apartamento|Residencial|        43|       43|      2|  null|        1|   1|    3|Paciência|Zona Oeste|       245|null|Venda|15000|
|085bab2c-87ad-452...|       Usado| Apartamento|Residencial|        42|       42|      2|  null|        1|   1|    2|Paciência|Zona Oeste|         0|   0|Venda|15000|
|18d22cbe-1b86-476...|       Usado| Apartamento|Residencial|        41|       41|      2|  null|        1|   1|    1|Guaratiba|Zona Oeste|         0|   0|Venda|20000

## Armazenamento de dados tratados
- Essa sessão se dedica a armazenar o dataset construído em dois tipos de formato: parquet e csv. A princípio somente um arquivo de armazenamento é suficiente para seguir com as demais análises, entretanto a ideia aqui é avaliar a diferença de desempenho entre os dois processos.

In [34]:
from time import time # função para fazer o tracking de tempo de execução

start = time()

dataset.write.parquet(
    '/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset-tratado/parquet' 
    ,mode = 'overwrite'
    )

end = time()

time_parquet_transcrition = end-start

In [35]:
start = time()

dataset.coalesce(1).write.csv(
    path='/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset-tratado/csv'
    ,header=True
    ,sep=';'
    ,mode='overwrite'
)

end = time()

time_csv_transcrition = end-start

In [36]:
print('Tempo de execução para transcrição em parquet: {:.3f} s'.format(time_parquet_transcrition))
print('Tempo de execução para transcrição em csv: {:.3f} s'.format(time_csv_transcrition))

Tempo de execução para transcrição em parquet: 8.227 s
Tempo de execução para transcrição em csv: 8.041 s


- Podemos notar que a transcrição da base tratada para os formatos parquet e csv leva aproximadamente o mesmo intervalo de tempo para se suceder. Mas e quanto à leitura dessa mesma base nos diferentes formatos?

In [37]:
start = time()

df_auxiliar = spark.read.csv(
    path='/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset-tratado/csv',
    header=True,
    inferSchema=True,
    sep=';'
)

end = time()

time_csv_reading = end - start

In [39]:
start = time()

df_auxiliar = spark.read.parquet(
    '/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset-tratado/parquet'
)

end = time()

time_parquet_reading = end -start

In [40]:
print('Tempo necessário para leitura da base de dados no formato parquet: {:.3f} s'.format(time_parquet_reading))
print('Tempo necessário para leitura da base de dados no formato csv: {:.3f} s'.format(time_csv_reading))

Tempo necessário para leitura da base de dados no formato parquet: 1.055 s
Tempo necessário para leitura da base de dados no formato csv: 1.836 s


- Seguimos não observando nenhuma grande diferença. A leitura do formato parquet foi cerca de 1s mais rápida (apenas) que a leitura do formato csv. Para nós no presente momento isso não faz muita diferença, mas o formato parquet é, de fato, mais otimizado e apropriado para lidar com grandes quantidades de informações e registros. Esses resultados seriam mais discrepantes se estivéssemos lidando com uma base de dados maior.